In [1]:
# System imports
import os

In [2]:
# External imports
import tensorflow as tf
import tensorflow.keras.layers as layers

## Load the dataset

Let's play with loading the dataset using `tf.data`

In [3]:
def _parse_data(sample_proto):
    parsed_example = tf.parse_single_example(
        sample_proto,
        features = {'3Dmap': tf.FixedLenFeature([], tf.string),
                    'unitPar': tf.FixedLenFeature([], tf.string),
                    'physPar': tf.FixedLenFeature([], tf.string)}
    )

    # Decode the data and normalize
    data = tf.decode_raw(parsed_example['3Dmap'], tf.float32)    
    data = tf.reshape(data, [128, 128, 128, 1])
    #data = tf.reshape(data, [128, 128, 128, 4])[:,:,:,0:1]
    data /= (tf.reduce_sum(data) / 128**3)
    
    # Decode the targets and normalize
    label = tf.decode_raw(parsed_example['unitPar'], tf.float32)
    
    return data, label

https://portal.nersc.gov/project/m3363/cosmoUniverse_2018_10_3parA/cosmoUniverse_2018_10_3parA.html

In [4]:
#data_dir = '/project/projectdirs/m3363/www/cosmoUniverse_2019_02_4parE/dim128_cube_nT4'
data_dir = '/project/projectdirs/m3363/www/cosmoUniverse_2018_10_3parA/dim128_cube'
#data_dir = '/global/cscratch1/sd/djbard/cosmoML/data-3param/13000-2xDupe'

In [5]:
n_train_files = None
samples_per_file = 16
input_shape = (128, 128, 128, 1)
target_size = 3

batch_size = 8
n_epochs = 4

In [6]:
train_data_dir = data_dir #os.path.join(data_dir, 'train')
train_files = [os.path.join(train_data_dir, f)
               for f in os.listdir(train_data_dir)
               if f.endswith('tfrecords')][:n_train_files]

In [7]:
n_train_files = len(train_files)
n_train = n_train_files * samples_per_file
print('Loaded %i training samples in %i TFRecord files' % (n_train, n_train_files))

Loaded 68480 training samples in 4280 TFRecord files


In [8]:
# Construct the data pipeline
train_dataset = (
    tf.data.TFRecordDataset(filenames=train_files)
    .map(_parse_data)
    .batch(batch_size)
)

### Debugging the dataset

## Build a Keras model

In [9]:
dropout = 0.5

In [10]:
model = tf.keras.models.Sequential()

model.add(layers.Conv3D(16, kernel_size=3, padding='same', input_shape=input_shape))
model.add(layers.LeakyReLU())
model.add(layers.AvgPool3D(pool_size=2))

model.add(layers.Conv3D(32, kernel_size=3, padding='same'))
model.add(layers.LeakyReLU())
model.add(layers.AvgPool3D(pool_size=2))

model.add(layers.Conv3D(64, kernel_size=3, padding='same'))
model.add(layers.LeakyReLU())
model.add(layers.AvgPool3D(pool_size=2))

model.add(layers.Conv3D(128, kernel_size=3, padding='same'))
model.add(layers.LeakyReLU())
model.add(layers.AvgPool3D(pool_size=2))

model.add(layers.Conv3D(256, kernel_size=3, padding='same'))
model.add(layers.LeakyReLU())
model.add(layers.AvgPool3D(pool_size=2))

model.add(layers.Flatten())
model.add(layers.Dense(1024))
model.add(layers.Dropout(dropout))
model.add(layers.LeakyReLU())

model.add(layers.Dense(512))
model.add(layers.Dropout(dropout))
model.add(layers.LeakyReLU())

model.add(layers.Dense(target_size))

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='mse', metrics=['mae'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 128, 128, 128, 16) 448       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 128, 16) 0         
_________________________________________________________________
average_pooling3d (AveragePo (None, 64, 64, 64, 16)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 64, 64, 64, 32)    13856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 64, 32)    0         
_________________________________________________________________
average_pooling3d_1 (Average (None, 32, 32, 32, 32)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 32, 32, 32, 64)    55360     
__________

In [12]:
steps_per_epoch = n_train // batch_size

In [13]:
history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=n_epochs)

Instructions for updating:
Use tf.cast instead.
Epoch 1/4
3822/8560 [============>.................] - ETA: 12:16 - loss: 0.3351 - mean_absolute_error: 0.4099

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
history.history

{'loss': [0.26377844097153674,
  3.0815238431254296e-05,
  3.599911031688586e-09,
  4.2055035416922733e-13],
 'mean_absolute_error': [0.3873832, 0.3873832, 0.3873832, 0.3873832]}